Generate data by running 

./sampler.py --density 0.1
./sampler.py --density 0.1 --odd 

to generate a datset in the folder 'data' with all tumbling rates. Now we have the data we want to train a model on it using the architecture designed. To do this run the following code:

In [ ]:
# Import modules
import numpy as np
import h5py
import glob
import re
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,BatchNormalization,AveragePooling2D,LeakyReLU,GlobalAveragePooling2D,ReLU

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.stats import pearsonr

np.set_printoptions(precision=3, suppress=True)

from tensorflow import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,BatchNormalization,AveragePooling2D,Normalization

### Import and prepare data

In [ ]:
# Define dataloading

def extract_floats(string):
    return re.findall(r"[-+]?\d*\.\d+|\d+", string)

def data_load():
    density = 0.15
    files = glob.glob(f"../data/dataset_tumble_*_0.1_1000.h5") #imports all tumbling rates for density 0.1 with 250 snapshots
    inputs,outputs = [],[]
    for f in files:
        tumble = float(extract_floats(f)[0])
        with h5py.File(f, "r") as fin:
          count = 0
          for key in fin.keys():
              img = fin[key][:]
              img = img.reshape((img.shape[0], img.shape[1],1))
              shape = img.shape
              inputs.append(img)
              outputs.append(tumble)
              count+=1

    # Scramble the dataset
    order = np.arange(len(outputs)).astype(int)
    order = np.random.permutation(order)
    return np.array(inputs)[order],np.array(outputs)[order],shape

In [ ]:
x,y,shape = data_load()

In [ ]:
# Set up test train split
last = int(np.shape(y)[0]*0.2) # 80% train data
x_train, y_train = x[:-last], y[:-last]
x_val,y_val = x[-last:],y[-last:]

print("Size of training data: ", len(x_train))
print("Size of validation data: ", len(x_val))

In [ ]:
# Set Up Options
import contextlib

@contextlib.contextmanager
def options(options):
  old_opts = tf.config.optimizer.get_experimental_options()
  tf.config.optimizer.set_experimental_options(options)
  try:
    yield
  finally:
    tf.config.optimizer.set_experimental_options(old_opts)

In [ ]:
# Run this after analysis to reset model and release RAM before changing the architecture
import gc

K.clear_session()


print("Collected: ", gc.collect())

In [ ]:
# Create Model Architecture

fixed_seed = 296 #choose seed (comment out if not needed)

if 'fixed_seed' in locals():
    keras.utils.set_random_seed(fixed_seed)
    print("Running program with fixed seed:",fixed_seed)
else:
    print("Running program with random seed.")

def make_net(shape):
    model = Sequential()

    model.add(Conv2D(filters=3, kernel_size=(3,3), padding='same', strides=(3,3), activation='relu', input_shape=shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=3, kernel_size=(3,3), padding='same'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(3, 3)))

    #model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters=6, kernel_size=(3,3), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=6, kernel_size=(3,3), padding='same'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Dense(units=128, activation='relu'))

    # with options({"layout_optimizer": False}):
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation='softmax'))

    model.add(Flatten())
    model.add(Dense(units=1, activation='linear'))
    return model

In [ ]:
model = make_net(shape)

model.summary()

In [ ]:
# Add optimiser

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Run training
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    verbose=True,
    batch_size=64,
    validation_data=(x_val, y_val)
)

In [ ]:
# Output learningCurve values if needed

# f = open("learningCurves.txt", "a")
# f.write("labelName") 
# for value in history.history["val_loss"]: f.write(","+str(value))  # Replace "val_loss" with "loss" for training loss
# f.write("\n")
# f.close()

In [ ]:
# Alternatively load pre-trained model (comment out previous cell to save time)

# name = "modelName"
# model = tf.keras.models.load_model(f'../models/{name}.keras')
# model.summary()

In [ ]:
# Analyse training results

prediction = model.predict(x_val, batch_size=64)
print("Shape of prediction : ", np.shape(prediction))

In [ ]:
# Save model

model.save("../models/modelName.keras")